In [ ]:
from google.colab import files
files.upload()  # Upload your CSV here

Saving synthetic_discount_dataset.csv to synthetic_discount_dataset.csv


{'synthetic_discount_dataset.csv': b'user_id,user_age,user_tier,cart_value,category,coupon_type,discount_amt,loyalty_pts_used,used_offer,final_price,purchased\nU00000,56,Free,2480.0,Home,bank_offer,316.0,0,1,2164.0,True\nU00001,46,Silver,1800.0,Grocery,bank_offer,263.0,150,1,1387.0,True\nU00002,32,Free,2490.0,Fashion,loyalty_discount,380.0,0,1,2110.0,True\nU00003,60,Silver,2640.0,Beauty,bank_offer,497.0,100,1,2043.0,True\nU00004,25,Free,2920.0,Home,none,0.0,0,0,2920.0,False\nU00005,38,Free,1420.0,Beauty,store_coupon,348.0,0,1,1072.0,True\nU00006,56,Free,3760.0,Beauty,bank_offer,517.0,50,1,3193.0,False\nU00007,36,Free,1710.0,Grocery,loyalty_discount,114.0,0,0,1596.0,False\nU00008,40,Free,1120.0,Electronics,none,0.0,0,0,1120.0,False\nU00009,28,Free,1940.0,Home,none,0.0,0,0,1940.0,False\nU00010,28,Silver,1680.0,Fashion,loyalty_discount,367.0,150,1,1163.0,False\nU00011,41,Free,1430.0,Fashion,none,0.0,0,0,1430.0,False\nU00012,53,Gold,1850.0,Electronics,loyalty_discount,331.0,400,1,1119.0,Tr

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load data
df = pd.read_csv('synthetic_discount_dataset.csv')

# Encode categorical columns
label_cols = ['user_tier', 'category', 'coupon_type']
le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col])

# Feature Engineering: Add derived features
df['discount_pct'] = df['discount_amt'] / df['cart_value']
df['loyalty_ratio'] = df['loyalty_pts_used'] / df['cart_value']

# Define features and target
X = df.drop(['user_id', 'purchased'], axis=1)
y = df['purchased'].astype(int)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:05:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.70      0.67      0.68      1059
           1       0.64      0.67      0.66       941

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000

ROC AUC: 0.7228647923421431


In [ ]:
def recommend_best_discount(user_row, possible_offers, model, le_coupon):
    best_offer = None
    best_prob = 0
    for coupon_type, discount_amt, loyalty_used in possible_offers:
        temp = user_row.copy()
        temp['coupon_type'] = le_coupon.transform([coupon_type])[0]
        temp['discount_amt'] = discount_amt
        temp['loyalty_pts_used'] = loyalty_used
        temp['final_price'] = max(temp['cart_value'] - discount_amt - loyalty_used, 0)
        temp['used_offer'] = int(discount_amt > 0 or loyalty_used > 0)
        temp['discount_pct'] = discount_amt / temp['cart_value']
        temp['loyalty_ratio'] = loyalty_used / temp['cart_value']

        features = temp.drop(['user_id', 'purchased'], errors='ignore')
        prob = model.predict_proba([features])[0][1]
        if prob > best_prob:
            best_prob = prob
            best_offer = (coupon_type, discount_amt, loyalty_used, prob)
    return best_offer


In [ ]:
# Select a random user row
sample_user = df.iloc[100].copy()

# Define hypothetical discount strategies
possible_offers = [
    ("none", 0, 0),
    ("bank_offer", 300, 0),
    ("store_coupon", 200, 100),
    ("loyalty_discount", 150, 150)
]

# Recommend best offer
best = recommend_best_discount(sample_user, possible_offers, model, le)
print(f"Best offer: {best}")


Best offer: ('bank_offer', 300, 0, np.float32(0.8230476))


In [ ]:
recommendations = []

for i in range(100):  # First 100 users in the dataset
    user_row = df.iloc[i].copy()
    best = recommend_best_discount(user_row, possible_offers, model, le)
    recommendations.append({
        "user_index": i,
        "recommended_coupon": best[0],
        "discount_amount": best[1],
        "loyalty_used": best[2],
        "predicted_prob": best[3]
    })

# Convert to DataFrame for easy viewing
recommendations_df = pd.DataFrame(recommendations)
print(recommendations_df.head(10))


   user_index recommended_coupon  discount_amount  loyalty_used  \
0           0   loyalty_discount              150           150   
1           1         bank_offer              300             0   
2           2   loyalty_discount              150           150   
3           3       store_coupon              200           100   
4           4   loyalty_discount              150           150   
5           5         bank_offer              300             0   
6           6       store_coupon              200           100   
7           7         bank_offer              300             0   
8           8         bank_offer              300             0   
9           9         bank_offer              300             0   

   predicted_prob  
0        0.566503  
1        0.793509  
2        0.749192  
3        0.636616  
4        0.627589  
5        0.905402  
6        0.615354  
7        0.801632  
8        0.982147  
9        0.671318  


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

# Import or define your recommend_best_discount, possible_offers, model, le here
# from your_module import recommend_best_discount, possible_offers, model, le

@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        user_data = request.json  # Input JSON sent by client
        user_row = pd.Series(user_data)  # Convert input dict to Pandas Series

        # Call your existing recommendation function
        best_offer = recommend_best_discount(user_row, possible_offers, model, le)

        response = {
            "recommended_coupon": best_offer[0],
            "discount_amount": best_offer[1],
            "loyalty_used": best_offer[2],
            "predicted_prob": best_offer[3]
        }
        return jsonify(response), 200
    except Exception as e:
        # Handle errors gracefully
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
